# Ungraded Lab: Improving Computer Vision Accuracy using Convolutions

## Shallow Neural Network

In the previous lessons, you saw how to do fashion recognition using a neural network containing three layers -- the input layer (in the shape of the data), the output layer (in the shape of the desired output) and only one hidden layer. You experimented with the impact of different sizes of hidden layer, number of training epochs etc on the final accuracy. For convenience, here's the entire code again. Run it and take a note of the test accuracy that is printed out at the end.

In [1]:
# Import all package I will need
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

plt.style.use('ggplot')

In [ ]:
# create dataset
(training_images, training_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()

print(f'shape of training_images: {training_images.shape}')
print(f'shape of training_labels: {training_labels.shape}')
print(f'shape of test_images: {test_images.shape}')
print(f'shape of test_labels: {test_labels.shape}')
